In [82]:
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Pretty display for notebooks
%matplotlib inline


In [83]:
#Parameters
data_directory = "/Fergus/Code/Data"
file1 = "/Airbnb/Dublin/summary_listings.csv"
file2 = "/Airbnb/Dublin/listings.csv"
file3 = "/Airbnb/Dublin/reviews.csv/reviews.csv"
name_of_dependent_variable = 'income'
skewed = ['capital-gain', 'capital-loss']

numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [84]:
#Open data file
data_summary = pd.read_csv(data_directory + file1)
data_extended = pd.read_csv(data_directory + file2)
data_reviews = pd.read_csv(data_directory + file3)

In [85]:
#EXPLORATION
#Display the first few records
display(data_summary.head(n=3))

data_summary.shape

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,14842,"Best location Portobello entire house, can sle...",58344,Hilary,NaN,Dublin City,53.331057,-6.269163,Entire home/apt,119,20,42,2019-01-02,0.46,1,5
1,44077,cosy comfortable Irish cottage twin,193005,Teresa,NaN,Dn Laoghaire-Rathdown,53.289950,-6.258126,Private room,50,3,182,2018-12-09,1.91,3,309
2,68483,Unique Central Dublin River-view,339817,John,NaN,Dublin City,53.345013,-6.265524,Entire home/apt,88,3,99,2018-11-06,1.04,1,84


(9520, 16)

In [23]:
print(data_extended.columns.tolist())

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

In [86]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
data_extended.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,14842,https://www.airbnb.com/rooms/14842,20190122134607,2019-01-22,"Best location Portobello entire house, can sle...","Beautiful entire house, newly revamped, with s...","Gorgeous, newly revamped central Dublin city e...","Beautiful entire house, newly revamped, with s...",none,Portobello is an exceptional place to live - v...,NaN,"There is great public transport - buses, Luas,...",Guests get the whole house to themselves.,I love meeting guests and I give you extensive...,"This house is a charming space, just completel...",NaN,NaN,https://a0.muscache.com/im/pictures/106656330/...,NaN,58344,https://www.airbnb.com/users/show/58344,Hilary,2009-11-29,"Dublin, County Dublin, Ireland",An easy-going and friendly Irish woman. Works ...,within an hour,100%,NaN,t,https://a0.muscache.com/im/users/58344/profile...,https://a0.muscache.com/im/users/58344/profile...,NaN,1,1,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,"Dublin, Dublin City, Ireland",Portobello,Dublin City,NaN,Dublin,Dublin City,Dublin 8,Dublin,"Dublin, Ireland",IE,Ireland,53.331057,-6.269163,f,House,Entire home/apt,3,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,Heating,""Family/kid ...",0.0,$119.00,NaN,NaN,$500.00,$60.00,2,$30.00,20,31,20,20,31,31,20.0,31.0,3 days ago,t,2,2,5,5,2019-01-22,42,16,2011-07-11,2019-01-02,95.0,10.0,9.0,10.0,10.0,10.0,9.0,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.46
1,44077,https://www.airbnb.com/rooms/44077,20190122134607,2019-01-22,cosy comfortable Irish cottage twin,Our house was built in 1937 when there was ple...,"Hi, My name is Teresa and we have 3 rooms ava...",Our house was built in 1937 when there was ple...,none,I like our neighbourhood as there is no shorta...,Dundrum was always known as a Spa Town and the...,There is a bus stop 2 mins up the road from us...,We have a beautiful landscaped garden with tab...,We live on the premises so we are always on ha...,Welcome to our cottage and we hope you will en...,NaN,NaN,https://a0.muscache.com/im/pictures/525706/050...,NaN,193005,https://www.airbnb.com/users/show/193005,Teresa,2010-08-06,Dublin Ireland,"I am a friendly outgoing Irishwoman ,who love...",within an hour,100%,NaN,t,https://a0.muscache.com/im/users/193005/profil...,https://a0.muscache.com/im/users/19

In [87]:

data_extended['host_neighbourhood'].value_counts(dropna=False)

NaN                                    3158
North City Central/O'Connell Street     497
Docklands                               455
Phibsborough                            402
The Liberties                           391
Ranelagh and Rathmines                  284
Ballsbridge                             258
Drumcondra                              244
Old City                                241
Stoneybatter/Arbour Hill                221
Trinity College                         207
Smithfield                              206
Georgian Quarter                        196
Donnybrook                              165
Grafton Street/St. Stephens Green       160
Portobello                              155
Rathgar                                 150
Kilmainham                              142
Temple Bar                              134
Cabra                                    92
Sandymount                               91
East Wall                                84
Clontarf                        

In [ ]:
#WHAT ARE THE MOST EXPENSIVE AREAS OF THE CITY TO PURCHASE?

#Look at the list of areas
#Get rid of ones with low frequency
#create grooups of these
#Then subdivide by accomadation type/num of rooms etc.
#Get average.

#Which part of town gets the best reviews for location? for transport?

In [118]:
#create a copy of the data_extended data.
data_areas = data_extended.copy()


In [78]:
#Clean up the data
#1. Remove rows that have NaN for neighbourhood
#2. Remove data from shared rooms as they are fairly infrequent and are likely to have quite different prices
#3. Remove data from neighbourhoods with only a small number of datapoints

#1. Remove rows that have NaN for neighbourhood
data_areas.dropna(subset=['host_neighbourhood'], inplace=True)

#2. Remove data from shared rooms
data_areas = data_areas[~data_areas['room_type'] == 'Shared_room']

#The min number of times the neighbourhood needs to feature in the list to be considered for comparision 
min_frequency = 9

#extract a list of neighbourhoods to ignore
greater_than_min_frequency = data_areas['host_neighbourhood'].value_counts().values < 9
ignore_these_neighbourhoods = list(data_areas['host_neighbourhood'].value_counts().index[greater_than_min_frequency])

print(ignore_these_neighbourhoods)

#get rid of the datapoints that are in the neigbourhoods to be ignored
data_areas_clean = data_areas[~data_areas['host_neighbourhood'].isin(ignore_these_neighbourhoods)]


['Terézváros - District VI.', 'Riverston Abbey', 'Dublin Industrial Estate', 'Donnycarney', 'Grace Park', 'De Pijp', 'Rose Bay/Dover Heights', 'Edenmore', 'Fulham', 'Kilmore', 'Woluwe-Saint-Lambert', 'French Riviera', 'Ixelles', 'Passy', 'Tour Eiffel - Champ de Mars', 'Downtown Montreal', 'Opéra - Grands Boulevards', 'Oud-West', 'Auteuil', 'Marine Parade', 'Richmond District', 'Saint-Germain-des-Prés - Odéon', 'MLK & 183', 'LB of Haringey', 'West Village']


In [81]:
data_areas_clean['room_type'].value_counts()

Entire home/apt    3599
Private room       2587
Shared room         131
Name: room_type, dtype: int64

In [54]:
#sort the listings into groups

data_area_groups = data_areas_clean[]

106

In [ ]:
data_extended.drop()

In [ ]:
#Create list

In [ ]:
#Next: Do a groupby first and then drop the awkard neighbourhoods

data_extended.groupby